In [1]:
!pip install pyspark
!pip install install-jdk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=98b89ed721d388ba801e778f1134320da8a4bc50ecd39503f2ab8868a451387a
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
#mengimport library untuk dataset ini
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.feature import HashingTF, Tokenizer, StopWordsRemover, IDF
from pyspark.ml import Pipeline
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [3]:
appName = "Penilaian otomatis menggunakan metode PySpark"
spark = SparkSession \
    .builder \
    .appName(appName) \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [4]:
#memuat dan membaca dataset
import pandas as pd
nilai_otomatis = pd.read_excel("/content/training_data_essay.xlsx")
nilai_otomatis

,npm,nama_peserta,jawaban,soal,skor_per_soal
0,0,Admin,"Tidak, Hanya membutuhkan satu karena satu soft...",1,100.00
1,0,Admin,Biaya dihitung berdasarkan waktu pengerjaan da...,2,100.00
2,0,Admin,Hak cipta adalah hak eksklusif bagi pencipta a...,3,100.00
3,0,Admin,Dijelaskan kepada klien jika huruf terlalu bes...,4,100.00
4,0,Admin,1. Melindungi dan menjamin pekerja atas hak ke...,5,100.00
...,...,...,...,...,...
115,1121020035,BS,posisi kepala dan leher harus tegak lurus deng...,8,48.41
116,1121020035,BS,kecocokan software camera dengan system hardwa...,9,65.89
117,1121020035,BS,okus dan exposure dipertimbangkan dalam operas...,10,48.51
118,1121020035,BS,peralatan yang dibutuhkan untuk transfer image...,11,78.41


In [5]:
#memuat dataframe dari dataset dan memilih kolomnya
df_dataset = spark.createDataFrame(nilai_otomatis)
df_dataset = df_dataset.select("soal", "jawaban", "skor_per_soal")
#menampilkan hasil
df_dataset.show()

+----+--------------------+-------------+
|soal|             jawaban|skor_per_soal|
+----+--------------------+-------------+
|   1|Tidak, Hanya memb...|        100.0|
|   2|Biaya dihitung be...|        100.0|
|   3|Hak cipta adalah ...|        100.0|
|   4|Dijelaskan kepada...|        100.0|
|   5|1. Melindungi dan...|        100.0|
|   6|Ruang Komputer, P...|        100.0|
|   7|Aturlah posisi pe...|        100.0|
|   8|Posisi Kepala dan...|        100.0|
|   9|1. Kecocokan soft...|        100.0|
|  10|1. Fokus dan expo...|        100.0|
|  11|1. Peralatan yang...|        100.0|
|  12|1. Dibuat grafik ...|        100.0|
|   1|tidak, cuma mengi...|         52.7|
|   2|biaya dihitung be...|        42.86|
|   3|hak membuat merup...|        42.16|
|   4|dipaparkan pada k...|        27.19|
|   5|1. mencegah serta...|        44.14|
|   6|ruang komputer, p...|        100.0|
|   7|aturlah posisi fi...|        57.68|
|   8|posisi kepala ser...|        45.71|
+----+--------------------+-------

In [6]:
#membersihkan teks dari kolom jawaban
#membuat pola untuk membersihkan teks
pola_reguler = "[^a-zA-Z\\s]+|\\d+"
#membersihkan kolom 'jawaban'
membersihkan_train = df_dataset.withColumn("jawaban_baru", regexp_replace("jawaban", pola_reguler, ""))
#menampilkan hasil
membersihkan_train.show()

+----+--------------------+-------------+--------------------+
|soal|             jawaban|skor_per_soal|        jawaban_baru|
+----+--------------------+-------------+--------------------+
|   1|Tidak, Hanya memb...|        100.0|Tidak Hanya membu...|
|   2|Biaya dihitung be...|        100.0|Biaya dihitung be...|
|   3|Hak cipta adalah ...|        100.0|Hak cipta adalah ...|
|   4|Dijelaskan kepada...|        100.0|Dijelaskan kepada...|
|   5|1. Melindungi dan...|        100.0| Melindungi dan m...|
|   6|Ruang Komputer, P...|        100.0|Ruang Komputer Po...|
|   7|Aturlah posisi pe...|        100.0|Aturlah posisi pe...|
|   8|Posisi Kepala dan...|        100.0|Posisi Kepala dan...|
|   9|1. Kecocokan soft...|        100.0| Kecocokan softwa...|
|  10|1. Fokus dan expo...|        100.0| Fokus dan exposu...|
|  11|1. Peralatan yang...|        100.0| Peralatan yang d...|
|  12|1. Dibuat grafik ...|        100.0| Dibuat grafik ya...|
|   1|tidak, cuma mengi...|         52.7|tidak cuma men

In [7]:
#membuat kolom jawan
pemisah_kata_dengan_koma = Tokenizer(inputCol = "jawaban_baru", outputCol = "jawaban_menggunakan_koma")
pemisah_kata_dengan_koma_train = pemisah_kata_dengan_koma.transform(membersihkan_train)
#menampilkan hasil
pemisah_kata_dengan_koma_train.show()

+----+--------------------+-------------+--------------------+------------------------+
|soal|             jawaban|skor_per_soal|        jawaban_baru|jawaban_menggunakan_koma|
+----+--------------------+-------------+--------------------+------------------------+
|   1|Tidak, Hanya memb...|        100.0|Tidak Hanya membu...|    [tidak, hanya, me...|
|   2|Biaya dihitung be...|        100.0|Biaya dihitung be...|    [biaya, dihitung,...|
|   3|Hak cipta adalah ...|        100.0|Hak cipta adalah ...|    [hak, cipta, adal...|
|   4|Dijelaskan kepada...|        100.0|Dijelaskan kepada...|    [dijelaskan, kepa...|
|   5|1. Melindungi dan...|        100.0| Melindungi dan m...|    [, melindungi, da...|
|   6|Ruang Komputer, P...|        100.0|Ruang Komputer Po...|    [ruang, komputer,...|
|   7|Aturlah posisi pe...|        100.0|Aturlah posisi pe...|    [aturlah, posisi,...|
|   8|Posisi Kepala dan...|        100.0|Posisi Kepala dan...|    [posisi, kepala, ...|
|   9|1. Kecocokan soft...|     

In [8]:
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql import SparkSession

#membuat spark session
spark = SparkSession.builder.appName("StopWordsExample").getOrCreate()
#membaca file stop words
with open('/content/id.stopwords.02.01.2016.txt', 'r') as f:
    stopwords = f.read().splitlines()
#membuat kolom yang ada kata pentingnya
hapus_sw = StopWordsRemover(inputCol='jawaban_menggunakan_koma', outputCol='kata_penting', stopWords=stopwords)
#menghapus stop words dari dataset
hapus_sw_train = hapus_sw.transform(pemisah_kata_dengan_koma_train)
#menampilkan hasil
hapus_sw_train.show()

+----+--------------------+-------------+--------------------+------------------------+--------------------+
|soal|             jawaban|skor_per_soal|        jawaban_baru|jawaban_menggunakan_koma|        kata_penting|
+----+--------------------+-------------+--------------------+------------------------+--------------------+
|   1|Tidak, Hanya memb...|        100.0|Tidak Hanya membu...|    [tidak, hanya, me...|[membutuhkan, sof...|
|   2|Biaya dihitung be...|        100.0|Biaya dihitung be...|    [biaya, dihitung,...|[biaya, dihitung,...|
|   3|Hak cipta adalah ...|        100.0|Hak cipta adalah ...|    [hak, cipta, adal...|[hak, cipta, hak,...|
|   4|Dijelaskan kepada...|        100.0|Dijelaskan kepada...|    [dijelaskan, kepa...|[klien, huruf, me...|
|   5|1. Melindungi dan...|        100.0| Melindungi dan m...|    [, melindungi, da...|[, melindungi, me...|
|   6|Ruang Komputer, P...|        100.0|Ruang Komputer Po...|    [ruang, komputer,...|[ruang, komputer,...|
|   7|Aturlah posis

In [9]:
#menghitung nilai hash dari kolom kata penting
fungsi_hash = hapus_sw_train.withColumn("nilai_hash", hash("kata_penting"))
fungsi_hash.show()

+----+--------------------+-------------+--------------------+------------------------+--------------------+-----------+
|soal|             jawaban|skor_per_soal|        jawaban_baru|jawaban_menggunakan_koma|        kata_penting| nilai_hash|
+----+--------------------+-------------+--------------------+------------------------+--------------------+-----------+
|   1|Tidak, Hanya memb...|        100.0|Tidak Hanya membu...|    [tidak, hanya, me...|[membutuhkan, sof...|-1678823266|
|   2|Biaya dihitung be...|        100.0|Biaya dihitung be...|    [biaya, dihitung,...|[biaya, dihitung,...|  868079823|
|   3|Hak cipta adalah ...|        100.0|Hak cipta adalah ...|    [hak, cipta, adal...|[hak, cipta, hak,...| 1708817134|
|   4|Dijelaskan kepada...|        100.0|Dijelaskan kepada...|    [dijelaskan, kepa...|[klien, huruf, me...| 1377676868|
|   5|1. Melindungi dan...|        100.0| Melindungi dan m...|    [, melindungi, da...|[, melindungi, me...| 1706744332|
|   6|Ruang Komputer, P...|     

In [10]:
#menghitung data training dan data testing
split = fungsi_hash.randomSplit([0.7, 0.3])
training = split[0].withColumnRenamed("skor_per_soal", "hasil_point")
testing = split[1].withColumnRenamed("skor_per_soal", "point_benar")
training_baris = training.count()
testing_baris = testing.count()
#menampilkan hasil
print("Hasil data training dari dataset ini:", training_baris)
print("Hasil data testing dari dataset ini:", testing_baris)

Hasil data training dari dataset ini: 87
Hasil data testing dari dataset ini: 33


In [11]:
#menampilkan dari dataframe
print("Dataframe di training dataset")
training.show()
print("\nDataframe di testing dataset")
testing.show()

Dataframe di training dataset
+----+--------------------+-----------+--------------------+------------------------+--------------------+-----------+
|soal|             jawaban|hasil_point|        jawaban_baru|jawaban_menggunakan_koma|        kata_penting| nilai_hash|
+----+--------------------+-----------+--------------------+------------------------+--------------------+-----------+
|   1|Tidak, Hanya memb...|      100.0|Tidak Hanya membu...|    [tidak, hanya, me...|[membutuhkan, sof...|-1678823266|
|   1|tidak, hanya memb...|      100.0|tidak hanya membu...|    [tidak, hanya, me...|[membutuhkan, sof...|-1678823266|
|   1|tidak, hanya memb...|      100.0|tidak hanya membu...|    [tidak, hanya, me...|[membutuhkan, sof...|-1678823266|
|   1|tidak, hanya memb...|      100.0|tidak hanya membu...|    [tidak, hanya, me...|[membutuhkan, sof...|-1678823266|
|   2|Biaya dihitung be...|      100.0|Biaya dihitung be...|    [biaya, dihitung,...|[biaya, dihitung,...|  868079823|
|   2|biaya dihitu

In [12]:
from pyspark.ml.recommendation import ALS

#menghitung als dan prediksi dari dataset ini
#menginisialisasi model ALS
hitung_als = ALS(maxIter=5, regParam=0.01, userCol="nilai_hash", itemCol="soal", ratingCol="hasil_point")
#melatih modelnya
model_als = hitung_als.fit(training)
#melakukan prediksinya
prediksi = model_als.transform(testing)
#menampilkan prediksinya
prediksi.select("soal", "nilai_hash", "point_benar", "prediction").show()

+----+-----------+-----------+----------+
|soal| nilai_hash|point_benar|prediction|
+----+-----------+-----------+----------+
|  12|-1557371155|      64.29|  64.28998|
|  12|  703135559|      81.06|       NaN|
|   1|   53244348|       52.7|       NaN|
|   6|  829849610|      100.0|  99.99994|
|   3| -348474399|      91.71|  91.70995|
|   3| -294850405|      83.43|  83.42996|
|   3|  638329998|      76.06|       NaN|
|   4|  636521394|      72.06|  72.40594|
|   8|-1927812291|      89.17|  89.16996|
|   8|-1861440661|      100.0|  99.99996|
|   8|  106718928|      45.71|       NaN|
|   7|-1894676254|      86.22|  86.21995|
|   7|-1894676254|      86.22|  86.21995|
|   7|-1894676254|      86.22|  86.21995|
|   7|  829849610|      86.22|-22.447596|
|  10|-1836299422|      94.51|       NaN|
|  10|-1238323704|      100.0|       NaN|
|  11| -871483067|      41.99|       NaN|
|  11| -521292198|      100.0|       NaN|
|  11| -383074426|      100.0|       NaN|
+----+-----------+-----------+----

In [13]:
#menghitung hasil MSE sebelum prediksi bersih
evaluasi_mse = RegressionEvaluator(metricName = "mse", labelCol = "point_benar", predictionCol = "prediction")
hasil_mse = evaluasi_mse.evaluate(prediksi)
#menghitung hasil root MSE sebelum prediksi bersih
evaluasi_rmse = RegressionEvaluator(metricName = "rmse", labelCol = "point_benar", predictionCol = "prediction")
hasil_rmse = evaluasi_rmse.evaluate(prediksi)
#menampilkan hasilnya
print("Hasil dari mean square error (MSE):", hasil_mse)
print("Hasil dari root mean square error (RMSE):", hasil_rmse)

Hasil dari mean square error (MSE): nan
Hasil dari root mean square error (RMSE): nan


In [14]:
#menghitung missing value
prediksi.count()
y = prediksi.count()
prediksi_bersih = prediksi.dropna(how = "any", subset = ["prediction"])
z = prediksi_bersih.count()
jumlah = y-z
#menampilkan hasil
# Menampilkan hasil
print("Jumlah missing value sebelum dihapus: {} baris".format(y))
print("Jumlah missing value sesudah dihapus: {} baris".format(z))
print("Total missing value: {} baris".format(jumlah))

Jumlah missing value sebelum dihapus: 33 baris
Jumlah missing value sesudah dihapus: 18 baris
Total missing value: 15 baris


In [15]:
#menghitung hasil MSE setelah prediksi bersih
hasil_mse = evaluasi_mse.evaluate(prediksi_bersih)
#menghitung hasil root MSE setelah prediksi bersih
hasil_rmse = evaluasi_rmse.evaluate(prediksi_bersih)
#menampilkan hasilnya
print("Hasil dari mean square error (MSE):", hasil_mse)
print("Hasil dari root mean square error (RMSE):", hasil_rmse)

Hasil dari mean square error (MSE): 656.0425558477425
Hasil dari root mean square error (RMSE): 25.613327699612608
